In [1]:
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import Cdo
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(sepl)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm
import metpy.constants as constants

In [ ]:
nino34 = np.random.randint(low=0, high=10, size=(36))
IMR = np.random.randint(low=0, high=10, size=(36))
hgt = np.random.randint(low=0, high=10, size=(36))

In [ ]:
linre_DJFnino34_sIndR = stats.linregress(nino34, IMR)
sIndR_GPCP_ENSO_related = linre_DJFnino34_sIndR[0]*nino34+linre_DJFnino34_sIndR[1]
sIndR_GPCP_ENSO_unrelated = IMR - sIndR_GPCP_ENSO_related
del(IMR)
# sIndR_GPCP_ENSO_related_nor = ca.standardize(sIndR_GPCP_ENSO_related)
# sIndR_GPCP_ENSO_unrelated_nor = ca.standardize(sIndR_GPCP_ENSO_unrelated)

In [ ]:
print(sIndR_GPCP_ENSO_related)

In [ ]:
stats.linregress(sIndR_GPCP_ENSO_related, hgt)

In [ ]:
stats.linregress(sIndR_GPCP_ENSO_unrelated, hgt)

In [ ]:
stats.linregress(nino34, hgt)

## 读取数据

In [2]:
fhfls_ssp585 = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/ssp585/mergetime_remap/hfls_UKESM1-0-LL_ssp585_remap.nc")
hfls_ssp585 = fhfls_ssp585["hfls"].sel(time=(fhfls_ssp585.time.dt.year>=2050)&(fhfls_ssp585.time.dt.year<=2099))*0.408*1e-6
hfls_ssp585.attrs["units"] = "mm/day"

fpr_ssp585 = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/ssp585/mergetime_remap/pr_UKESM1-0-LL_ssp585_remap.nc")
pr_ssp585 = fpr_ssp585["pr"].sel(time=(fhfls_ssp585.time.dt.year>=2050)&(fhfls_ssp585.time.dt.year<=2099))
pr_ssp585.attrs["units"] = "mm/day"

fhus_ssp585 = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/ssp585/mergetime_remap/hus_UKESM1-0-LL_ssp585_remap.nc")
hus_ssp585 = fhus_ssp585["hus"].sel(time=(fhfls_ssp585.time.dt.year>=2050)&(fhfls_ssp585.time.dt.year<=2099))

fua_ssp585 = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/ssp585/mergetime_remap/ua_UKESM1-0-LL_ssp585_remap.nc")
ua_ssp585 = fua_ssp585["ua"].sel(time=(fhfls_ssp585.time.dt.year>=2050)&(fhfls_ssp585.time.dt.year<=2099))

fva_ssp585 = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/ssp585/mergetime_remap/va_UKESM1-0-LL_ssp585_remap.nc")
va_ssp585 = fva_ssp585["va"].sel(time=(fhfls_ssp585.time.dt.year>=2050)&(fhfls_ssp585.time.dt.year<=2099))

fps_ssp585 = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/ssp585/mergetime_remap/ps_UKESM1-0-LL_ssp585_remap.nc")
ps_ssp585 = fps_ssp585["ps"].sel(time=(fhfls_ssp585.time.dt.year>=2050)&(fhfls_ssp585.time.dt.year<=2099))

fwap_ssp585 = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/ssp585/mergetime_remap/wap_UKESM1-0-LL_ssp585_remap.nc")
wap_ssp585 = fwap_ssp585["wap"].sel(time=(fhfls_ssp585.time.dt.year>=2050)&(fhfls_ssp585.time.dt.year<=2099))

In [3]:
fhfls_hist = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/hist/mergetime_remap/hfls_UKESM1-0-LL_hist_remap.nc")
hfls_hist = fhfls_hist["hfls"].sel(time=(fhfls_hist.time.dt.year>=1965)&(fhfls_hist.time.dt.year<=2014))*0.408*1e-6
hfls_hist.attrs["units"] = "mm/day"

fpr_hist = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/hist/mergetime_remap/pr_UKESM1-0-LL_hist_remap.nc")
pr_hist = fpr_hist["pr"].sel(time=(fhfls_hist.time.dt.year>=1965)&(fhfls_hist.time.dt.year<=2014))
pr_hist.attrs["units"] = "mm/day"

fhus_hist = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/hist/mergetime_remap/hus_UKESM1-0-LL_hist_remap.nc")
hus_hist = fhus_hist["hus"].sel(time=(fhfls_hist.time.dt.year>=1965)&(fhfls_hist.time.dt.year<=2014))

fua_hist = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/hist/mergetime_remap/ua_UKESM1-0-LL_hist_remap.nc")
ua_hist = fua_hist["ua"].sel(time=(fhfls_hist.time.dt.year>=1965)&(fhfls_hist.time.dt.year<=2014))

fva_hist = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/hist/mergetime_remap/va_UKESM1-0-LL_hist_remap.nc")
va_hist = fva_hist["va"].sel(time=(fhfls_hist.time.dt.year>=1965)&(fhfls_hist.time.dt.year<=2014))

fps_hist = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/hist/mergetime_remap/ps_UKESM1-0-LL_hist_remap.nc")
ps_hist = fps_hist["ps"].sel(time=(fhfls_hist.time.dt.year>=1965)&(fhfls_hist.time.dt.year<=2014))

fwap_hist = xr.open_dataset("/home/ys17-23/Extension2/LuoHL/EmerCon/hist/mergetime_remap/wap_UKESM1-0-LL_hist_remap.nc")
wap_hist = fwap_hist["wap"].sel(time=(fhfls_hist.time.dt.year>=1965)&(fhfls_hist.time.dt.year<=2014))

In [4]:
ua_ssp585_JJA = ca.p_time(ua_ssp585, 6, 8)
va_ssp585_JJA = ca.p_time(va_ssp585, 6, 8)
wap_ssp585_JJA = ca.p_time(wap_ssp585, 6, 8)
hus_ssp585_JJA = ca.p_time(hus_ssp585, 6, 8)

hfls_ssp585_JJA = ca.p_time(hfls_ssp585, 6, 8)
pr_ssp585_JJA = ca.p_time(pr_ssp585, 6, 8)
ps_ssp585_JJA = ca.p_time(ps_ssp585, 6, 8)

In [5]:
ua_hist_JJA = ca.p_time(ua_hist, 6, 8)
va_hist_JJA = ca.p_time(va_hist, 6, 8)
wap_hist_JJA = ca.p_time(wap_hist, 6, 8)
hus_hist_JJA = ca.p_time(hus_hist, 6, 8)

hfls_hist_JJA = ca.p_time(hfls_hist, 6, 8)
pr_hist_JJA = ca.p_time(pr_hist, 6, 8)
ps_hist_JJA = ca.p_time(ps_hist, 6, 8)

In [6]:
pr_prime_JJA = pr_ssp585_JJA.mean(dim="time")-pr_hist_JJA.mean(dim="time")
hfls_prime_JJA = hfls_ssp585_JJA.mean(dim="time")-hfls_hist_JJA.mean(dim="time")

In [7]:
hus_prime_JJA = hus_ssp585_JJA.mean(dim="time")-hus_hist_JJA.mean(dim="time")
ua_prime_JJA = ua_ssp585_JJA.mean(dim="time")-ua_hist_JJA.mean(dim="time")
va_prime_JJA = va_ssp585_JJA.mean(dim="time")-va_hist_JJA.mean(dim="time")
wap_prime_JJA = wap_ssp585_JJA.mean(dim="time")-wap_hist_JJA.mean(dim="time")

In [8]:
ua_bar_JJA = ua_hist_JJA.mean(dim="time")
va_bar_JJA = va_hist_JJA.mean(dim="time")
hus_bar_JJA = hus_hist_JJA.mean(dim="time")
wap_bar_JJA = wap_hist_JJA.mean(dim="time")

In [9]:
ps_bar_JJA = ps_hist_JJA.mean(dim="time")

In [10]:
husprime_dp, husprime_dy, husprime_dx = mpcalc.gradient(hus_prime_JJA)
husprime_dp = husprime_dp.metpy.dequantify()
husprime_dy = husprime_dy.metpy.dequantify()
husprime_dx = husprime_dx.metpy.dequantify()

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/pint/quantity.py:1313: RuntimeWarning: invalid value encountered in divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [11]:
husprime_dx.loc[85000, 25, 60]

<xarray.DataArray ()>
array(1.39711148e-09)
Coordinates:
    longitude  float64 60.0
    latitude   float64 25.0
    plev       float64 8.5e+04
Attributes:
    units:    1 / meter

In [12]:
husbar_dp, husbar_dy, husbar_dx = mpcalc.gradient(hus_bar_JJA)
husbar_dp = husbar_dp.metpy.dequantify()
husbar_dy = husbar_dy.metpy.dequantify()
husbar_dx = husbar_dx.metpy.dequantify()

In [13]:
ptop = 10000
g = 9.8
cal_level = hus_hist_JJA.coords["plev"].where(hus_hist_JJA.coords["plev"] >= ptop)
cal_level.attrs["units"] = "Pa"
CMIPdp = geocat.comp.dpres_plevel(cal_level, ps_bar_JJA, ptop)
CMIPdpg = CMIPdp / g
CMIPdpg.attrs["units"] = "kg/m2"

integration_top = 10000


def vert_intergrate(da_sl, CMIP6dpg=CMIPdpg, integration_top=integration_top, cal_level=cal_level):
  res_vert = (da_sl * CMIPdpg[:list(cal_level).index(integration_top)+1, :, :].data).sum(dim="plev")
  return(res_vert)


In [14]:
term1_1 = vert_intergrate(ua_bar_JJA.loc[:integration_top, :, :]*husprime_dx.loc[:integration_top, :, :].data)
term1_2 = vert_intergrate(va_bar_JJA.loc[:integration_top, :, :]*husprime_dy.loc[:integration_top, :, :].data)
term1 = term1_1+term1_2

In [15]:
term2_1 = vert_intergrate(ua_prime_JJA.loc[:integration_top, :, :]*husbar_dx.loc[:integration_top, :, :].data)
term2_2 = vert_intergrate(va_prime_JJA.loc[:integration_top, :, :]*husbar_dy.loc[:integration_top, :, :].data)
term2 = term2_1+term2_2

In [16]:
term3 = vert_intergrate(wap_bar_JJA.loc[:integration_top, :, :]*husprime_dp.loc[:integration_top, :, :].data)

In [17]:
term4 = vert_intergrate(wap_prime_JJA.loc[:integration_top, :, :]*husbar_dp.loc[:integration_top, :, :].data)

In [18]:
Res = pr_prime_JJA-hfls_prime_JJA-term1-term2-term3-term4

In [19]:
def cal_area_weighted_mean(da):
    """计算数据的区域（纬度加权）平均

    Args:
        da (dataarray): 原数据

    Returns:
        dataarray: 区域平均结果
    """    
    weights = np.cos(np.deg2rad(da.latitude))
    weights.name = "weights"
    da_mean = da.weighted(weights).mean(("latitude"), skipna=True)
    da_mean = da_mean.mean(dim="longitude", skipna=True)
    return da_mean

In [20]:
lat = hus_bar_JJA.coords["latitude"]
lon = hus_bar_JJA.coords["longitude"]

India_N = 24.0
India_S = 10.0
India_W = 60.0
India_E = 105.0
lat_India_range = lat[(lat >= India_S) & (lat <= India_N)]
lon_India_range = lon[(lon >= India_W) & (lon <= India_E)]


pr_prime_JJA_India_JJA = cal_area_weighted_mean(pr_prime_JJA.sel(latitude=lat_India_range, longitude=lon_India_range))
hfls_prime_JJA_India_JJA = cal_area_weighted_mean(hfls_prime_JJA.sel(latitude=lat_India_range, longitude=lon_India_range))
term1_India_JJA = cal_area_weighted_mean(term1.sel(latitude=lat_India_range, longitude=lon_India_range))
term2_India_JJA = cal_area_weighted_mean(term2.sel(latitude=lat_India_range, longitude=lon_India_range))
term3_India_JJA = cal_area_weighted_mean(term3.sel(latitude=lat_India_range, longitude=lon_India_range))
term4_India_JJA = cal_area_weighted_mean(term4.sel(latitude=lat_India_range, longitude=lon_India_range))
Res_India_JJA = cal_area_weighted_mean(Res.sel(latitude=lat_India_range, longitude=lon_India_range))

In [21]:
(pr_prime_JJA_India_JJA-hfls_prime_JJA_India_JJA-term1_India_JJA-term2_India_JJA-term3_India_JJA-term4_India_JJA)*86400
# term2_India_JJA

<xarray.DataArray ()>
array(1.42677805)
Attributes:
    standard_name:  precipitation_flux
    long_name:      Precipitation
    units:          mm/day
    comment:        includes both liquid and solid phases
    original_name:  mo: (stash: m01s05i216, lbproc: 128)
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    history:        2019-04-18T14:30:41Z altered by CMOR: replaced missing va...

In [22]:
print(pr_prime_JJA_India_JJA.data*86400)
print(hfls_prime_JJA_India_JJA.data*86400)
print(term1_India_JJA.data*86400)
print(term2_India_JJA.data*86400)
print(term3_India_JJA.data*86400)
print(term4_India_JJA.data*86400)

1.0733897630463343
0.37047973653053834
0.15509345966294522
0.1493765465096345
-1.3112677941303001
0.28292976596136


In [23]:
print(pr_prime_JJA.sel(latitude=20, longitude=40).data)
print(hfls_prime_JJA.sel(latitude=20, longitude=40).data)
print(term1.sel(latitude=20, longitude=40).data)
print(term2.sel(latitude=20, longitude=40).data)
print(term3.sel(latitude=20, longitude=40).data)
print(term4.sel(latitude=20, longitude=40).data)

7.227271e-08
4.5781817e-06
3.83943199271268e-06
-5.931632346535587e-06
4.036708279318352e-06
-4.837056696715183e-06


In [24]:
pr_prime_JJA.sel(latitude=20, longitude=40)-hfls_prime_JJA.sel(latitude=20, longitude=40)-term1.sel(latitude=20, longitude=40)-term2.sel(latitude=20, longitude=40)-term3.sel(latitude=20, longitude=40)-term4.sel(latitude=20, longitude=40)

<xarray.DataArray ()>
array(-1.61336038e-06)
Coordinates:
    longitude  float64 40.0
    latitude   float64 20.0
Attributes:
    standard_name:  precipitation_flux
    long_name:      Precipitation
    units:          mm/day
    comment:        includes both liquid and solid phases
    original_name:  mo: (stash: m01s05i216, lbproc: 128)
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    history:        2019-04-18T14:30:41Z altered by CMOR: replaced missing va...